In [37]:
! pip install keras-tuner

In [38]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [39]:
print(tf.__version__)

2.5.0


In [40]:
fashion_mnist = keras.datasets.fashion_mnist

In [41]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [42]:
# Train Test split
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [43]:
# Normalize the value this has to be done only for Train images
train_images = train_images/255.0
train_labels = train_labels/255.0

In [44]:
train_images[0].shape

(28, 28)

In [45]:
len(train_labels)

60000

In [46]:
# Reshaping the image data into 4D
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [47]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters = hp.Int('conv_1_filter', min_value = 16, max_value = 64, step = 16),
          # During one step an amount of batch_size examples (pictures or row) are processed.
          kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
          #the kernel is nothing but a filter that is used to extract the features from the images.
          #The kernel is a matrix that moves over the input data, performs the dot product with the sub-region of input data, and gets the output as the matrix of dot products.
          activation = 'relu',
          input_shape = (28,28,1)# 1 is for grey scale
        ),
          keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=8, max_value=16, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu'
      ),
          keras.layers.Flatten(),
          #Flatten is the function that converts the pooled 
          #feature map to a single column that is passed to the fully connected layer. Dense adds the fully connected layer to the neural network
          keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
          #Dense layer is the regular deeply connected neural network layer. It is most common and frequently used layer.
          #Dense layer does the below operation on the input and return the output.
  
      ),
          keras.layers.Dense(10, activation='softmax')])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [48]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [49]:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [50]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [51]:
model=tuner_search.get_best_models(num_models=1)[0]

In [52]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 48)        1248      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                819264    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 859,594
Trainable params: 859,594
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 5/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 6/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 8/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 9/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0994 - val_loss: 0.0000e+00 - val_accuracy: 0.1050
Epoch 10/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0000e+00 